In [1]:
from dataset import LFWDataset
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import numpy as np
from PIL import Image
import os

In [2]:
data = LFWDataset(
    faces_folder="../data/faces",
    smiling_labels_file="../data/SMILE_list.txt",
    non_smiling_labels_file="../data/NON-SMILE_list.txt",
    transform=transforms.Compose([
        transforms.Resize((64, 64)),
        transforms.ToTensor()
    ])
)

In [ ]:
# Indices
smiling_idx = [i for i, (_, label) in enumerate(data) if label == 0]
non_smiling_idx = [i for i, (_, label) in enumerate(data) if label == 1]

2